In [1]:
# ProdImage GenAI TextGen SEO CTA Twitter Keywords

In [ ]:
# Model Overview:
# 1.	Product image (retail products) was passed as part of the API call and the prompt was asked to return a description
# 2.	The short description returned was then further improved by another API call with a prompt
# 3.	Different prompt API calls were made to then extract different types of keywords
# 4.	Finally, HTML tags with all information was produced


In [2]:
#! pip install --upgrade openai torch transformers google-cloud-bigquery google-cloud-storage pandas numpy

In [3]:
import openai
import os
import base64
import requests

print(openai.__version__)

1.35.13


In [4]:
# Set your OpenAI API key
API_KEY = ''

# Define the OpenAI API URL
API_URL = "https://api.openai.com/v1/chat/completions"

# Function to generate image description using GPT-4
def generate_image_short_description(image_path):
    try:
        # Open and encode image to base64
        with open(image_path, 'rb') as image_file:
            image_bytes = image_file.read()
            image_base64 = base64.b64encode(image_bytes).decode('utf-8')

        headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {API_KEY}"
        }
        
        payload = {
          "model": "gpt-4o",
          "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "What’s in this image?"
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{image_base64}"
                  }
                }
              ]
            }
          ],
          "max_tokens": 300
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        short_description = response.json()['choices'][0]['message']['content'].strip()
        return short_description
    except Exception as e:
        print(f"Error in generating short description: {str(e)}")
        return None

# Function to generate longer description from images short_description using GPT-4
def generate_expanded_description(short_description):
    try:
        # Create the prompt to expand the description
        prompt = f"Expand the following short description into a longer, more detailed text: \"{short_description}\""

        # Define the headers and payload for the API request
        headers = {
            'Authorization': f'Bearer {API_KEY}',
            'Content-Type': 'application/json'
        }
        payload = {
            "model": "gpt-4",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant that expands short descriptions into detailed, longer text."},
                {"role": "user", "content": prompt}
            ]
        }

        # Make the API request
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()

        # Extract the expanded description from the response
        expanded_description = response.json()['choices'][0]['message']['content'].strip()
        return expanded_description

    except Exception as e:
        print(f"Error in generating expanded description: {str(e)}")
        return None

# Function to extract SEO keywords from a description
def extract_SEOkeywords(expanded_description):
    try:
        # Create the prompt to extract SEO keywords
        prompt = f"Extract SEO optimization keywords from the following text: \"{expanded_description}\""

        # Define the headers and payload for the API request
        headers = {
            'Authorization': f'Bearer {API_KEY}',
            'Content-Type': 'application/json'
        }
        payload = {
            "model": "gpt-4",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant that extracts SEO optimization keywords from text."},
                {"role": "user", "content": prompt}
            ]
        }

        # Make the API request
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()

        # Extract the keywords from the response
        response_SEOkeywords = response.json()['choices'][0]['message']['content'].strip()
        return response_SEOkeywords
    except Exception as e:
        print(f"Error in extracting SEO keywords: {str(e)}")
        return None

# Function to extract CTA keywords using age profile from a description        
def extract_CTAkeywords_ageprofile(description, age_profile):
    try:
        prompt = f"Generate a call-to-action (CTA) based on the following description and age profile. Description: \"{description}\" Age Profile: {age_profile}"

        headers = {
            'Authorization': f'Bearer {API_KEY}',
            'Content-Type': 'application/json'
        }
        payload = {
            "model": "gpt-4",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant that generates call-to-action (CTA) messages tailored to different age profiles."},
                {"role": "user", "content": prompt}
            ]
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()

        response_CTAkeywords = response.json()['choices'][0]['message']['content'].strip()
        return response_CTAkeywords
    except Exception as e:
        print(f"Error in generating CTA: {str(e)}")
        return None

# Function to generate meta tags
def generate_meta_tags(expanded_description,short_description,SEO_keywords,CTA_keywords,tone):
    
    meta_tags = {
        'summary': short_description,
        'description': expanded_description,
        'tone': tone,
        'SEO Keywords': SEO_keywords,
        'CTA Keywords': CTA_keywords,
        'twitter card': short_description[:70] + '...' if len(short_description) > 70 else short_description,
    }
    return meta_tags


In [5]:
# Function to process images and create HTML with meta tags
def process_images(image_directory, output_directory, age):
    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(image_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            print('Generating image description keywords using OpenAI gpt-4o model API: ') 
            image_path = os.path.join(image_directory, filename)
            short_description = generate_image_short_description(image_path)            
            expanded_description = generate_expanded_description(short_description)            
            SEO_keywords = extract_SEOkeywords(expanded_description)

            tone = ''
            if age < 26:
                age_profile = "Millennials"
                tone = 'Cool'
            elif 27 <= age <= 42:
                age_profile = "Gen Z"
                tone = 'Stylish'
            elif 43 <= age <= 58:
                age_profile = "Gen X"
                tone = 'Elegant'
            else:
                age_profile = "Baby Boomers"
                tone = 'Comfortable'
            
            CTA_keywords = extract_CTAkeywords_ageprofile(expanded_description, age_profile)
            
            if expanded_description:
                meta_tags = generate_meta_tags(expanded_description,short_description,SEO_keywords,CTA_keywords,tone)

                # Generate HTML webpage with description, keywords, twitter card for each image
                with open(image_path, 'rb') as img_file:
                    image_base64 = base64.b64encode(img_file.read()).decode('utf-8')

                html_content = f"""
                <html>
                <head>
                    <meta name="summary" content="{meta_tags['summary']}">
                    <meta name="tone" content="{meta_tags['tone']}">
                </head>
                <body style="text-align: center;">
                    <div style="margin: 0 auto; max-width: 600px;">
                        <h1>{meta_tags['summary']}</h1>
                        <img src="data:image/jpeg;base64,{image_base64}" alt="Image">
                        <p><strong>Description:</strong> {meta_tags['description']}</p>
                        <p><strong>SEO Keywords:</strong> {meta_tags['SEO Keywords']}</p>
                        <p><strong>CTA Keywords:</strong> {meta_tags['CTA Keywords']}</p>
                        <p><strong>twitter card:</strong> {meta_tags['twitter card']}</p>
                        <button>{meta_tags['CTA Keywords']}</button>
                    </div>
                </body>
                </html>
                """
                output_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}.html")
                with open(output_path, "w") as file:
                    file.write(html_content)
            print('Writing HTML file with image description keywords twitter card: ') 


In [6]:
# Run the process
process_images('/ProdImages', '/ProdHTML', age=40)


Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-4o model API: 
Writing HTML file with image description keywords twitter card: 
Generating image description keywords using OpenAI gpt-